In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

import keras as k
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

import cv2
from tqdm import tqdm

x_train = []
x_test = []
y_train = []

input_loc = 'input/'

df_train = pd.read_csv(input_loc + 'train.csv')

flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

for f, tags in tqdm(df_train.values, miniters=1000):
    img = cv2.imread(input_loc + 'train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train.append(cv2.resize(img, (32, 32)))
    y_train.append(targets)
    
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float16) / 255.

print(x_train.shape)
print(y_train.shape)

split = 35000
x_train, x_valid, y_train, y_valid = x_train[:split], x_train[split:], y_train[:split], y_train[split:]

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(32, 32, 3)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))

model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
              optimizer='adam',
              metrics=['accuracy'])
              
model.fit(x_train, y_train,
          batch_size=128,
          epochs=4,
          verbose=1,
          validation_data=(x_valid, y_valid))
          
from sklearn.metrics import fbeta_score

p_valid = model.predict(x_valid, batch_size=128)
print(y_valid)
print(p_valid)
print(fbeta_score(y_valid, np.array(p_valid) > 0.2, beta=2, average='samples'))

100%|██████████| 40479/40479 [02:19<00:00, 289.44it/s]


(40479, 32, 32, 3)
(40479, 17)
Train on 35000 samples, validate on 5479 samples
Epoch 1/4
35000/35000 [==============================] - 102s - loss: 0.2504 - acc: 0.9052 - val_loss: 0.1949 - val_acc: 0.9218
Epoch 2/4
35000/35000 [==============================] - 95s - loss: 0.1991 - acc: 0.9221 - val_loss: 0.1781 - val_acc: 0.9298
Epoch 3/4
35000/35000 [==============================] - 95s - loss: 0.1858 - acc: 0.9276 - val_loss: 0.1684 - val_acc: 0.9331
Epoch 4/4
35000/35000 [==============================] - 94s - loss: 0.1779 - acc: 0.9304 - val_loss: 0.1649 - val_acc: 0.9348
[[0 0 0 ..., 1 0 0]
 [1 0 1 ..., 0 0 0]
 [0 0 1 ..., 0 0 1]
 ..., 
 [0 0 1 ..., 0 0 0]
 [1 0 1 ..., 0 0 0]
 [0 0 1 ..., 0 1 0]]
[[  2.66329587e-01   7.71156920e-04   5.32044113e-01 ...,   8.39801412e-03
    1.92299426e-01   2.11154774e-01]
 [  3.11532617e-01   7.09707965e-04   5.48959613e-01 ...,   9.06245597e-03
    3.13218653e-01   2.43422747e-01]
 [  9.48599651e-02   2.06798330e-04   2.20757380e-01 ...,  